Классификация


Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

Загрузка и первичный анализ данных

In [2]:
df_cancer = pd.read_csv('cancer-risk-factors.csv')
print("Размер датасета:", df_cancer.shape)
print("\nПервые 5 строк:")
print(df_cancer.head())

Размер датасета: (2000, 21)

Первые 5 строк:
  Patient_ID Cancer_Type  Age  Gender  Smoking  Alcohol_Use  Obesity  \
0     LU0000      Breast   68       0        7            2        8   
1     LU0001    Prostate   74       1        8            9        8   
2     LU0002        Skin   55       1        7           10        7   
3     LU0003       Colon   61       0        6            2        2   
4     LU0004        Lung   67       1       10            7        4   

   Family_History  Diet_Red_Meat  Diet_Salted_Processed  ...  \
0               0              5                      3  ...   
1               0              0                      3  ...   
2               0              3                      3  ...   
3               0              6                      2  ...   
4               0              6                      3  ...   

   Physical_Activity  Air_Pollution  Occupational_Hazards  BRCA_Mutation  \
0                  4              6                     3    

Удаляем столбцы

In [3]:
columns_to_drop = ['Overall_Risk_Score', 'Risk_Level', 'Patient_ID']
df_cancer = df_cancer.drop(columns=columns_to_drop)
print("\nСтолбцы после удаления:")
print(df_cancer.columns.tolist())


Столбцы после удаления:
['Cancer_Type', 'Age', 'Gender', 'Smoking', 'Alcohol_Use', 'Obesity', 'Family_History', 'Diet_Red_Meat', 'Diet_Salted_Processed', 'Fruit_Veg_Intake', 'Physical_Activity', 'Air_Pollution', 'Occupational_Hazards', 'BRCA_Mutation', 'H_Pylori_Infection', 'Calcium_Intake', 'BMI', 'Physical_Activity_Level']


Разделяем признаки и целевую переменную

In [4]:
y = df_cancer['Cancer_Type']
X = df_cancer.drop(columns=['Cancer_Type'])

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (2000, 17)
y shape: (2000,)


Кодируем целевую переменную

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Классы рака:", label_encoder.classes_)


Классы рака: ['Breast' 'Colon' 'Lung' 'Prostate' 'Skin']


Разделяем данные

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)


Обучаем Logistic Regression

In [21]:
baseline_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('lr', LogisticRegression(max_iter=100))
])

baseline_pipeline.fit(X_train, y_train)


c:\Users\Kirill\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('imputer', ...), ('lr', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'mean'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a feature

Предсказание

In [22]:
y_pred_baseline = baseline_pipeline.predict(X_test)

Оценка качества

In [23]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))

Baseline Accuracy: 0.725

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.76      0.77      0.77        92
       Colon       0.71      0.69      0.70        84
        Lung       0.80      0.85      0.82       105
    Prostate       0.69      0.66      0.67        61
        Skin       0.57      0.55      0.56        58

    accuracy                           0.72       400
   macro avg       0.71      0.70      0.70       400
weighted avg       0.72      0.72      0.72       400



Улучшение бейзлайна. (масштабирование + подбор гиперпараметров)


Масштабирование численных признаков

In [24]:
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_features),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_features)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('lr', LogisticRegression(max_iter=100
    ))
])

Подбор гиперпараметров

In [25]:
param_grid = {
    'lr__C': [0.01, 0.1, 1, 10],
    'lr__penalty': ['l2'],  # l1 можно добавить, но нужно solver='liblinear'
    'lr__solver': ['lbfgs']
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

grid.fit(X_train, y_train)
print("Лучшие параметры:", grid.best_params_)

Лучшие параметры: {'lr__C': 1, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}


c:\Users\Kirill\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


Модель с улучшенным бейзлайном

In [26]:
best_lr = grid.best_estimator_
y_pred_improved = best_lr.predict(X_test)



Оценка качества

In [27]:
print("Improved Accuracy:", accuracy_score(y_test, y_pred_improved))
print("\nImproved Classification Report:")
print(classification_report(
    y_test,
    y_pred_improved,
    target_names=label_encoder.classes_
))

Improved Accuracy: 0.75

Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.79      0.77      0.78        92
       Colon       0.74      0.76      0.75        84
        Lung       0.80      0.86      0.83       105
    Prostate       0.74      0.69      0.71        61
        Skin       0.60      0.57      0.58        58

    accuracy                           0.75       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.75      0.75      0.75       400



Было

In [28]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))

Baseline Accuracy: 0.725

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.76      0.77      0.77        92
       Colon       0.71      0.69      0.70        84
        Lung       0.80      0.85      0.82       105
    Prostate       0.69      0.66      0.67        61
        Skin       0.57      0.55      0.56        58

    accuracy                           0.72       400
   macro avg       0.71      0.70      0.70       400
weighted avg       0.72      0.72      0.72       400



Выводы:

Accuracy, Weighted F1, Macro F1 выросли. 

Гипотезы о необходимости масштабирования и подбора гиперпараметров полностью подтвердились.

Имплементация алгоритма машинного обучения


In [29]:
class MyLogisticRegression:
    def __init__(self, lr=0.01, n_iter=500, fit_intercept=True):
        self.lr = lr
        self.n_iter = n_iter
        self.fit_intercept = fit_intercept

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        n_samples, n_features = X.shape

        if self.fit_intercept:
            X = np.hstack([np.ones((n_samples, 1)), X])

        self.theta = np.zeros(X.shape[1])

        # One-vs-rest для многоклассовой
        self.classes_ = np.unique(y)
        self.thetas_ = []

        for c in self.classes_:
            y_binary = (y == c).astype(float)
            theta = np.zeros(X.shape[1])
            for _ in range(self.n_iter):
                z = X.dot(theta)
                h = self._sigmoid(z)
                grad = X.T.dot(h - y_binary) / n_samples
                theta -= self.lr * grad
            self.thetas_.append(theta)

    def predict(self, X):
        X = np.array(X)
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
        scores = np.column_stack([self._sigmoid(X.dot(theta)) for theta in self.thetas_])
        return np.argmax(scores, axis=1)

Обучение и подготовка данных(только числовые)

In [30]:
X_num = X[numeric_features]
X_num = SimpleImputer(strategy='mean').fit_transform(X_num)

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_num, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

my_lr = MyLogisticRegression(lr=0.01, n_iter=500)
my_lr.fit(X_train_n, y_train_n)
y_pred_my = my_lr.predict(X_test_n)

Оценка качества

In [31]:
print("MyLogisticRegression Baseline Accuracy:", accuracy_score(y_test_n, y_pred_my))
print("\nMyLogisticRegression Baseline Classification Report:")
print(classification_report(
    y_test_n,
    y_pred_my,
    target_names=label_encoder.classes_
))

MyLogisticRegression Baseline Accuracy: 0.595

MyLogisticRegression Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       1.00      0.03      0.06        92
       Colon       0.59      0.81      0.68        84
        Lung       0.70      0.95      0.81       105
    Prostate       0.39      0.69      0.50        61
        Skin       0.78      0.43      0.56        58

    accuracy                           0.59       400
   macro avg       0.69      0.58      0.52       400
weighted avg       0.71      0.59      0.53       400



Было у алгоритма из sklearn


In [32]:
print("Baseline Accuracy:", accuracy_score(y_test, y_pred_baseline))
print("\nBaseline Classification Report:")
print(classification_report(
    y_test,
    y_pred_baseline,
    target_names=label_encoder.classes_
))

Baseline Accuracy: 0.725

Baseline Classification Report:
              precision    recall  f1-score   support

      Breast       0.76      0.77      0.77        92
       Colon       0.71      0.69      0.70        84
        Lung       0.80      0.85      0.82       105
    Prostate       0.69      0.66      0.67        61
        Skin       0.57      0.55      0.56        58

    accuracy                           0.72       400
   macro avg       0.71      0.70      0.70       400
weighted avg       0.72      0.72      0.72       400



Выводы: у алгоритма из slkearn сильно лучше метрики из-за ограниченного числа итераций и фиксированного learning rate, отсутствия обработки категориальных признаков.

Улучшение MyLogisticRegression

In [33]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

my_lr_improved = MyLogisticRegression(lr=0.01, n_iter=500)
my_lr_improved.fit(X_train_s, y_train_s)
y_pred_my_improved = my_lr_improved.predict(X_test_s)


Оценка


In [34]:
print("MyLogisticRegression Improved Accuracy:", accuracy_score(y_test_s, y_pred_my_improved))
print("\nMyLogisticRegression Improved Classification Report:")
print(classification_report(
    y_test_s,
    y_pred_my_improved,
    target_names=label_encoder.classes_
))

MyLogisticRegression Improved Accuracy: 0.7125

MyLogisticRegression Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.70      0.83      0.76        92
       Colon       0.72      0.74      0.73        84
        Lung       0.71      0.90      0.79       105
    Prostate       0.76      0.62      0.68        61
        Skin       0.67      0.24      0.35        58

    accuracy                           0.71       400
   macro avg       0.71      0.67      0.66       400
weighted avg       0.71      0.71      0.69       400



Было у улучшенного бейзлайна из sklearn

In [35]:
print("Improved Accuracy:", accuracy_score(y_test, y_pred_improved))
print("\nImproved Classification Report:")
print(classification_report(
    y_test,
    y_pred_improved,
    target_names=label_encoder.classes_
))

Improved Accuracy: 0.75

Improved Classification Report:
              precision    recall  f1-score   support

      Breast       0.79      0.77      0.78        92
       Colon       0.74      0.76      0.75        84
        Lung       0.80      0.86      0.83       105
    Prostate       0.74      0.69      0.71        61
        Skin       0.60      0.57      0.58        58

    accuracy                           0.75       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.75      0.75      0.75       400



Вывод: Результаты у собственной реализации хуже  чем у алгоритма из sklearn  после применения масштабирования и одинаковых гиперпараметров. Это произошло из-за ограниченного числа итераций и фиксированного learning rate, отсутствия обработки категориальных признаков.

Регрессия


Загрузка и первичный анализ данных

In [36]:
df_productivity = pd.read_csv('social_media_vs_productivity.csv')
print("Размер датасета:", df_productivity.shape)
print("\nПервые 5 строк:")
print(df_productivity.head())

Размер датасета: (30000, 19)

Первые 5 строк:
   age  gender    job_type  daily_social_media_time  \
0   56    Male  Unemployed                 4.180940   
1   46    Male      Health                 3.249603   
2   32    Male     Finance                      NaN   
3   60  Female  Unemployed                      NaN   
4   25    Male          IT                      NaN   

  social_platform_preference  number_of_notifications  work_hours_per_day  \
0                   Facebook                       61            6.753558   
1                    Twitter                       59            9.169296   
2                    Twitter                       57            7.910952   
3                   Facebook                       59            6.355027   
4                   Telegram                       66            6.214096   

   perceived_productivity_score  actual_productivity_score  stress_level  \
0                      8.040464                   7.291555           4.0   
1       

Обработка пропусков и уменьшения размера датасета


In [77]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

df_productivity_clean = df_productivity.dropna(
    subset=['actual_productivity_score']
)

df_productivity_clean = df_productivity_clean.iloc[:1000].copy()




Разделяем признаки и целевую переменную

In [78]:
target = 'actual_productivity_score'

X = df_productivity_clean.drop(columns=[target])
y = df_productivity_clean[target]


Препроцессинг



In [79]:
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)


Train / Test split

In [80]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Baseline Linear Regression

In [81]:
from sklearn.linear_model import LinearRegression, Ridge

baseline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('lr', LinearRegression())
])

baseline_lr.fit(X_train, y_train)
y_pred_baseline = baseline_lr.predict(X_test)

Оценка качества

In [82]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))

Baseline MSE: 0.2711304927974215
Baseline R2: 0.9197484648145187


Улучшение бейзлайна(масштабирование + регуляризация Ridge)

In [83]:
pipe_reg = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),  # масштабирование
    ('ridge', Ridge())  # регуляризация
])

Подбор гиперпараметров

In [84]:
param_grid = {
    'ridge__alpha': [0.01, 0.1, 1,8 ,9, 11, 50, 100]
}

grid_reg = GridSearchCV(
    pipe_reg,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_reg.fit(X_train, y_train)
print("Лучшие параметры:", grid_reg.best_params_)

Лучшие параметры: {'ridge__alpha': 9}


Оценка улучшенного бейзлайна

In [85]:
best_ridge = grid_reg.best_estimator_
y_pred_improved = best_ridge.predict(X_test)

print("Improved MSE:", mean_squared_error(y_test, y_pred_improved))
print("Improved R2:", r2_score(y_test, y_pred_improved))

Improved MSE: 0.27361716448797724
Improved R2: 0.9190124383402907


Было

In [86]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))

Baseline MSE: 0.2711304927974215
Baseline R2: 0.9197484648145187


Выводы: после проверки гипотез значимой разницы не было обнаружено. Для задачи регрессии линейная модель уже является оптимальной аппроксимацией данных.
Дополнительные гипотезы (масштабирование, регуляризация) не приводят к заметному улучшению качества.

Собственная реализация Linear Regression

In [87]:
class MyLinearRegression:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
        self.coef_ = np.linalg.pinv(X.T @ X) @ (X.T @ y)

    def predict(self, X):
        X = np.array(X)
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
        return X @ self.coef_

Подготовка данных для MyLinearRegressor (используем только числовые признаки)




In [88]:
X_num = SimpleImputer(strategy='mean').fit_transform(X[numeric_features])
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_num, y, test_size=0.2, random_state=42
)

Обучение и оценка

In [89]:
my_lr = MyLinearRegression()
my_lr.fit(X_train_n, y_train_n)
y_pred_my = my_lr.predict(X_test_n)

print("MyLinearRegression Baseline MSE:", mean_squared_error(y_test_n, y_pred_my))
print("MyLinearRegression Baseline R2:", r2_score(y_test_n, y_pred_my))

MyLinearRegression Baseline MSE: 0.27139987082883477
MyLinearRegression Baseline R2: 0.9196687319879261


Было у алгоритма из sklearn

In [90]:
print("Baseline MSE:", mean_squared_error(y_test, y_pred_baseline))
print("Baseline R2:", r2_score(y_test, y_pred_baseline))

Baseline MSE: 0.2711304927974215
Baseline R2: 0.9197484648145187


Вывод: собственная реализация оказалась практически такой же, что подтверждает корректность реализации


Используем гипотезы для собственного алгоритма

In [91]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

my_lr_improved = MyLinearRegression()
my_lr_improved.fit(X_train_s, y_train_s)
y_pred_my_improved = my_lr_improved.predict(X_test_s)

print("MyLinearRegression Improved MSE:", mean_squared_error(y_test_s, y_pred_my_improved))
print("MyLinearRegression Improved R2:", r2_score(y_test_s, y_pred_my_improved))

MyLinearRegression Improved MSE: 0.27139987082884526
MyLinearRegression Improved R2: 0.919668731987923


Было у улучшенного бейзлайна из sklearn

In [92]:
best_ridge = grid_reg.best_estimator_
y_pred_improved = best_ridge.predict(X_test)

print("Improved MSE:", mean_squared_error(y_test, y_pred_improved))
print("Improved R2:", r2_score(y_test, y_pred_improved))

Improved MSE: 0.27361716448797724
Improved R2: 0.9190124383402907


Вывод: Результаты у улучшенной собственной реализации оказались идентичнымы улучшенному алгоритма из sklearn, что подтверждает корректность реализации.